In [1]:
#DEPENDENCIES
#######################################################################################################################
import sys
sys.path.insert(0, '..')
from functions_object_detection import *
from functions_line_detection import *
from config.tesseract import path_to_my_tesseract
from functions_word_detection import *
from functions_char_detection import *
from cut_to_characters import *
from utils.util_miscellaneous import changeDirectory

#dont require pip
import os
import shutil
import string
from os import listdir
from os.path import isfile, join

#require pip
import cv2
import numpy as np
import pandas as pd
import docx
import pytesseract
import re

######################################################################################################################################
#pytesseract.pytesseract.tesseract_cmd = 'C:\\Program Files\\Tesseract-OCR\\tesseract.exe'
pytesseract.pytesseract.tesseract_cmd = path_to_my_tesseract
#custom_config = r'--oem 3 --psm 6'
custom_config = r'-l por+eng --psm 6'

In [2]:
#inputs
parent_directory = os.path.abspath('')
root = os.path.abspath(os.path.join(parent_directory, os.pardir))

data_input_folder = os.path.join(root, 'test', 'data', 'input')
data_output_folder = os.path.join(root, 'test', 'data', 'output')


#clean and create paths to save images after preprocessing
create_path(os.path.join(data_output_folder, 'filtered', 'output_all_br')) #source image preprocessing output with blured backround to compare
create_path(os.path.join(data_output_folder, 'filtered', 'output_all')) #source image preprocessing output ready to exctract objects and lines
create_path(os.path.join(data_output_folder, 'recognition')) # extracted objects, lines, and classification results


#read all the source images for recognition
#sourcepath='.\\data\\\\'
sourcepath=data_input_folder
photos = [f for f in listdir(sourcepath) if isfile(join(sourcepath, f))]


# Object detection

In [3]:
dictionary=[]
for p in photos:
    print(p)

    #create a path for each image to store the results of text objects, text lines and the word file with the classification output.
    path_objs=os.path.join(data_output_folder, 'recognition', 'objects\\'+p+'\\')
    print('path_objs', path_objs, '\\')
    #path_objs_original= '.\\app_output\\recognition\\objects\\'+p+'\\' #path to text objects
    #print('path_objs_original', path_objs_original)
    create_path(path_objs)

    path_lines=os.path.join(data_output_folder, 'recognition', 'lines\\'+p+'\\') #path to objext text lines
    #path_lines='.\\app_output\\recognition\\lines\\'+p+'\\' #path to objext text lines
    create_path(path_lines)

    path_words=os.path.join(data_output_folder, 'recognition', 'words\\'+p+'\\') 
    #path_words='.\\app_output\\recognition\\words\\'+p+'\\'
    create_path(path_words)

    path_letters=os.path.join(data_output_folder, 'recognition', 'letters\\'+p+'\\') 
    #path_letters='.\\app_output\\recognition\\letters\\'+p+'\\'
    create_path(path_letters)

    path_quant=os.path.join(data_output_folder, 'recognition', 'quantized\\'+p+'\\') 
    #path_quant='.\\app_output\\recognition\\quantized\\'+p+'\\'
    create_path(path_quant)
        
        
    
    img=cv2.imread(os.path.join(data_input_folder, p))
    #img=cv2.imread(os.path.join()'.\\data\\'+p)

    thresh,gray=preprocess_orig_img(img)
    horizontal_p1=mask_morph(get_horizodal_lines(thresh.copy(),25),thresh.copy())
    vertical=mask_morph(get_vertical_lines(thresh.copy(),50),horizontal_p1.copy()) #20
    image_wide=wide_white_contours(vertical.copy(),int(img.shape[0]*expot(vertical.copy())),int(img.shape[1] * expot(vertical.copy())))
    noise_removed=remove_noise(image_wide.copy())  
    masked_img_n, masked_img_weighted_big_n,masked_img_nf, masked_img_weighted_big_nf,masked_img_nfs, masked_img_weighted_big_nfs,        filter1_nf=extract_contoured_image_1(img.copy(), noise_removed.copy(),0,0,0.2,0.8)
    masked_imgf2,masked_img_weighted_bigf2,filter1_f2=extract_contoured_image_2(img.copy(), filter1_nf.copy(),0,0,0.2,0.8)
    write_images_pre(masked_imgf2,masked_img_weighted_bigf2,p)

    lines,temp_object= extract_object(img.copy(),filter1_f2,p,path_objs)

    if not 'base_images' in temp_object:
        temp_object['base_images'] = []
    
    temp_object['base_images'].append(img)

    if not 'image_name' in temp_object:
        temp_object['image_name'] = []
    
    temp_object['image_name'].append(p)

            
    ##Line detection
    ###################################################################################################################
    idl=0 #id for line per image
    idw=0 #id for word per image
    idq=0 #quantized image
    idt=0 #text letter
    mydoc_tesseract = docx.Document()
    for key in range (0,len(temp_object['line_images'])):
        print('line image', key)
        #get an object image
        img_line=temp_object['line_images'][key]
        
        #preprocess it
        try:
            img_score,img_res=line_preprocessing(img_line, 31,29,19,17,30,0,0,4,3,'two')  
            print('Base Image')
        except:
            img_score,img_res=line_preprocessing_advanced(img_line) 
            print('Base Image v2 / can be deleted')
            
        #get all the lines per object and line pixels write them on a path
        line_images,pixels,idl=line_detection(img_score,img_res,path_lines,idl)
        print('line detection pixels', pixels)
    
        #1 text output per maximum hierarchy
        text=[]
        
        #recognition
        for line in range (0,len(line_images)):
            #get all the words for complicated objects that lines cant be identified
            if check_for_complicated_lines(0.2,0.85,10,line_preprocessing(line_images[line],13,9,0,0,30,2,0,0,0,'one'))==True:
                words_images,words_pixels,idw=word_detection(idw,path_words,line_preprocessing(line_images[line],13,9,0,0,30,2,0,3,3,'one'))
                           
                for w in range(0, len(words_images)):
                    idq+=1
                    text_tesseract=tesseract_recognition(words_images[w],path_to_my_tesseract,False,20)
                    mydoc_tesseract.add_paragraph(text_tesseract)
                    text.append(text_tesseract)
                    
                    #start letter recognition
                    colors_count_sorted,color_arr,backround_id =get_color_ids(words_images[w])
                    image_color_decr=colors_decrease_minkowski_v2(color_arr,backround_id,words_images[w],1.5)
                    quant1=quantizate_colors(image_color_decr,14,2)
                    quant2=quantizate_colors(quant1,2,30)
                    colors_count_sorted,color_array,backround_id=get_color_ids(quant2)
                    image_underscore=check_for_underscore(quant2,color_array)
                    image_hough= horizodal_lines_remove_Hough(image_underscore,backround_id,0.15,0.05)
                    cv2.imwrite(path_quant+str(idq) + '.png', quant2)
                    colordictioary, color_array,backround_id=get_color_ids(image_hough)
                    colors_first_color_index,colors_last_color_index,colors_count_by_column_all_colors=count_occur_color_by_row_and_column(image_hough,color_array)
                    try:
                        y_dif=abs(colors_first_color_index[colors_first_color_index>0].min()-colors_last_color_index[colors_last_color_index>0].max())
                        #returns letter pixels by line word
                        idt,letter_pixels,letter_images=cut_to_characters(words_images[w],image_hough,colors_count_by_column_all_colors,colors_first_color_index,colors_last_color_index,idt,path_letters)
                    except:
                        #support filtering of symbols
                        letter_pixels=[]
                        print('icon')
                    #aggregate the pixels with lines     
                    pixels[line]=pixels[line]+[words_pixels[w]+[letter_pixels]]
            else:
                text_tesseract=tesseract_recognition(line_images[line],path_to_my_tesseract,False,20)
                mydoc_tesseract.add_paragraph(text_tesseract)
                text.append(text_tesseract) 
                #start letter recognition
                
                colors_count_sorted,color_arr,backround_id =get_color_ids(line_images[line])
                image_color_decr=colors_decrease_minkowski_v2(color_arr,backround_id,line_images[line],1.5)
                quant1=quantizate_colors(image_color_decr,14,2)
                quant2=quantizate_colors(quant1,2,30)
                colors_count_sorted,color_array,backround_id=get_color_ids(quant2)
                image_underscore=check_for_underscore(quant2,color_array)
                image_hough= horizodal_lines_remove_Hough(image_underscore,backround_id,0.15,0.05)
                cv2.imwrite(path_quant+str(idq) + '.png', quant2)
                colordictioary, color_array,backround_id=get_color_ids(image_hough)

            
                colors_first_color_index,colors_last_color_index,colors_count_by_column_all_colors=count_occur_color_by_row_and_column(image_hough,color_array)
                try:
                    y_dif=abs(colors_first_color_index[colors_first_color_index>0].min()-colors_last_color_index[colors_last_color_index>0].max())
                    #returns letter pixels by line
                    idt,letter_pixels,letter_images=cut_to_characters(line_images[line],image_hough,colors_count_by_column_all_colors,colors_first_color_index,colors_last_color_index,idt,path_letters)
                except Exception as e:
                    #helps to reduce symbols
                    letter_pixels=[]
                    print(str(e))
                    
                print('pixels line', pixels[line])
                print('[letter_pixels]', [letter_pixels])
                pixels[line]=pixels[line]+[letter_pixels]

        temp_object['ocr_text'][key]=text
        print('key is', key)

        temp_object['sub_coordinates'][key]=pixels

    #mydoc_tesseract.save(path_lines+"OCR_output_teserract.docx")
    mydoc_tesseract.save(os.path.join(path_lines, 'OCR_output_teserract.docx'))
    print('lines recogized for  image:',p)
            
#save the object
np.save(os.path.join(data_output_folder, 'object_output.npy'), temp_object) 

ex_001.png
path_objs c:\Users\JosePombo\Desktop\repos\ocr\test\data\output\recognition\objects\ex_001.png\ \
written done
(455, 58)
line image 0
Base Image
<<<<<<<<<<<<<<<<<<<<<<<<<<<<<line_detection>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>><<
gonna add pixels to line_pixel -> [0, 15]
line detection pixels [[0, 15]]
1063
pixels line [0, 15]
[letter_pixels] [[[0, 15, 5, 10], [0, 15, 10, 16], [0, 15, 16, 22], [0, 15, 22, 28]]]
key is 0
line image 1
Base Image
<<<<<<<<<<<<<<<<<<<<<<<<<<<<<line_detection>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>><<
gonna add pixels to line_pixel -> [0, 15]
line detection pixels [[0, 15]]
1062
pixels line [0, 15]
[letter_pixels] [[[0, 15, 5, 10], [0, 15, 10, 16], [0, 15, 16, 22], [0, 15, 22, 28]]]
key is 1
line image 2
Base Image
<<<<<<<<<<<<<<<<<<<<<<<<<<<<<line_detection>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>><<
gonna add pixels to line_pixel -> [0, 15]
line detection pixels [[0, 15]]
1061
pixels line [0, 15]
[letter_pixels] [[[0, 15, 5, 10], [0, 15, 10, 16], [0, 15, 16, 22], [0, 

In [ ]:
# estrutura de dados
result = [{'objBbox': [], 'text': '', 'chars': [{'charBbox': [], 'char': ''}] }]

for imgIdx in range(0, len(temp_object['base_images'])):
    print('imgIdx', imgIdx)
    img = temp_object['base_images'][imgIdx]
    annot_img = img.copy()
    image_name = temp_object['image_name'][imgIdx]
    image_annotation_folder = os.path.join(data_output_folder, 'images_annotated', image_name)
    create_path(image_annotation_folder)

    for objectBboxIdx in range (0, len(temp_object['coordinates_line'])):
        objectBbox = temp_object['coordinates_line'][objectBboxIdx]  #coordinates in the form of y0, y1, x0, x1
        #print('objectBbox', objectBbox)

        snippet = img[objectBbox[0]: objectBbox[1], objectBbox[2]: objectBbox[3]]
        cv2.imwrite(os.path.join(image_annotation_folder, "snippet_" + str(objectBboxIdx) + ".png"), snippet)

        for coordIdx in range (0,len(temp_object['sub_coordinates'][0][0][2:][0])): #coordinates in the form of y0, y1, x0, x1
            charBbox = temp_object['sub_coordinates'][objectBboxIdx][0][2:][0][coordIdx]
            #print('charBbox', charBbox)
            y0 = charBbox[0]
            y1 = charBbox[1]
            x0 = charBbox[2]
            x1 = charBbox[3]

            relBbox = [y0, y1, x0, x1]
            #print('relBbox',relBbox)

            snippet_char = snippet[y0: y1, x0: x1]

            snippet_char_path = "snippet_" + str(objectBboxIdx) + "char_" + str(coordIdx) + ".png"
            cv2.imwrite(os.path.join(image_annotation_folder, "snippet_" + str(objectBboxIdx) + "_char_" + str(coordIdx) + ".png"), snippet_char)
            
            y0 = objectBbox[0] + charBbox[0]
            y1 = objectBbox[0] + charBbox[1]
            x0 = objectBbox[2] + charBbox[2]
            x1 = objectBbox[2] + charBbox[3]
            absBbox = [y0, y1, x0, x1]
            #print('absBbox', absBbox)
            
            cv2.rectangle(annot_img, (x0, y0), (x1, y1), (255, 0, 0), 1)

    image_annotated_path = os.path.join(image_annotation_folder, 'annotation.png')
    print('image_annotated_path', image_annotated_path)
    cv2.imwrite(image_annotated_path, annot_img)

    
            # a seguir preciso de invocar o char recognizer p/ cada caracter
           # recognizer = os.path.join()
        
        #aqui concateno todo os chars reconhecidos na propriedade 'text'
        
        '''
        p/ terça 02/11

        necessário pôr o recognition a reconhecer snippets passados como image array e meter loops em funcoes
        '''
        

In [ ]:
#https://iq-inc.com/importerror-attempted-relative-import/

In [ ]:
import recogntition

In [ ]:
np.save('object_output.npy', temp_object) 

In [ ]:
import numpy as np
read_dictionary = np.load('object_output.npy',allow_pickle='TRUE').item()
print(read_dictionary)